In [27]:
import pandas
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from dbfread import DBF
import numpy as np
import shutil
import tpot
from glob import glob


In [30]:
def open_dbf_as_frame(dbf_path, encoding = "UTF-8"):
    """Opens a DBF as a pandas dataframe"""
    dbf = DBF(dbf_path, encoding)
    return pandas.DataFrame(iter(dbf))


def find_changes(change_directory):
    """Returns a list of dbf files that contain changes"""
    out = []
    with os.scandir(change_directory) as it:
        for file in it:
            if file.endswith(".dbf"):
                if check_for_real_change(file):
                    out.append(file)


def check_for_real_change(dbf_path):
    frame = open_dbf_as_frame(dbf_path)


def normalise_column(dataframe, column, min_val=0, max_val=1):
    floats = dataframe[column].values.astype(float)
    scaler = preprocessing.MinMaxScalar()
    scaled = scalar.fit_transform(floats)
    dataframe.loc[:,column] = scaled
    

In [16]:
changes_frame = open_dbf_as_frame(r"D:\conafor\changes\changes.dbf")
#false_frame =pandas.DataFrame([open_dbf_as_frame(frame) for frame in glob(r"D:\CONAFOR\false changes\*.dbf")])
false_frame = open_dbf_as_frame(r"D:\CONAFOR\false changes\Jal_LCC_IRE_LANDSAT2014_2016_PARTE2.dbf")

In [23]:
foo = "maf_2"
changes_frame[foo]

0      -4.915037
1      -2.152306
2      -0.762475
3      -6.188069
4      -5.330953
5       5.247676
6       0.754436
7      -2.944607
8       2.959903
9       4.276427
10      7.535928
11     -3.617070
12     -1.740002
13      0.014006
14     -1.561229
15      3.258401
16     -0.538015
17      0.336020
18      0.575550
19     -0.278190
20      1.623141
21      0.493700
22     -4.199460
23     -0.958318
24      0.311273
25     -0.161764
26      0.036454
27     -0.413286
28      0.264839
29      0.242770
          ...   
5574   -0.511274
5575   -2.548631
5576   -1.752843
5577   -1.131555
5578   -2.196405
5579   -1.572842
5580   -0.905038
5581   -0.525744
5582   -0.733409
5583   -1.757665
5584   -1.060178
5585   -1.328657
5586   -0.809264
5587   -0.618995
5588   -0.361330
5589   -1.362891
5590   -0.187874
5591   -0.333082
5592   -0.734261
5593   -0.469655
5594   -1.766482
5595   -0.952193
5596    1.481912
5597    0.302044
5598   -1.781410
5599   -2.109811
5600   -0.664127
5601   -0.8739

In [12]:
false_frame.columns

Index([u'Join_Count', u'TARGET_FID', u'OBJECTID_1', u'FID_LCC_IR', u'oid_1',
       u'change', u'mdmx_t1', u'mdmx_t2', u'mdmx_t1t2', u'mdmx_cls',
       u'ipcc_dyna', u'mdmx_t1_l', u'prob_cmb', u'maf_4', u'maf_5', u'maf_6',
       u'maf_1', u'maf_2', u'maf_3', u'tcd_t2', u'tcd_t1', u'met_t1_1',
       u'met_t1_3', u'met_t1_2', u'met_t1_5', u'met_t1_4', u'met_t1_7',
       u'met_t1_6', u'met_t2_6', u'met_t2_7', u'met_t2_4', u'met_t2_5',
       u'met_t2_2', u'met_t2_3', u'met_t2_1', u'area', u'id', u'DNDVIT2_T1',
       u'Val_Cambio', u'Shape_Leng', u'Shape_Area', u'MASC', u'interprete',
       u'metodo', u'Val_T1', u'Val_T2', u'Join_Cou_1', u'TARGET_F_1',
       u'LC_16'],
      dtype='object')

In [12]:
feature_columns = [u'tcd_t1', u'met_t1_1',
                   u'met_t1_3', u'met_t1_2', u'met_t1_5', u'met_t1_4', u'met_t1_7',
                   u'met_t1_6', u'met_t2_6', u'met_t2_7', u'met_t2_4', u'met_t2_5',
                   u'met_t2_2', u'met_t2_3', u'met_t2_1', u'Shape_Area', 'DNDVIT2_T1']

In [13]:
change_features = changes_frame.filter(items=feature_columns).sample(2000).values
false_features = changes_frame.filter(items=feature_columns).sample(2000).values
features = np.concatenate([change_features, false_features])

In [31]:
normalise_column(changes_frame, 'met_t1_1')

AttributeError: 'module' object has no attribute 'MinMaxScalar'

In [21]:
classes = np.concatenate(
    [np.ones(len(change_features)), np.zeros(len(false_features))])

In [22]:
print(classes.shape)
print(features.shape)

(4000L,)
(4000L, 18L)


In [23]:
train_features, test_features, train_classes, test_classes = train_test_split(features, classes)

In [24]:
train_classes

array([0., 1., 1., ..., 0., 0., 1.])

In [25]:
optimiser = tpot.TPOTClassifier(generations = 500, population_size = 100, n_jobs=-1, cv=5, verbosity=2, warm_start=True)
optimiser.fit(train_features, train_classes)
optimiser.export("conafor_tpot_v7.py")

SEJveChjaGlsZHJlbj0oSW50UHJvZ3Jlc3ModmFsdWU9MCwgZGVzY3JpcHRpb249dSdPcHRpbWl6YXRpb24gUHJvZ3Jlc3MnLCBtYXg9NTAxMDAsIHN0eWxlPVByb2dyZXNzU3R5bGUoZGVzY3LigKY=


Generation 1 - Current best internal CV score: 0.514342241691
Generation 2 - Current best internal CV score: 0.514342241691
Generation 3 - Current best internal CV score: 0.514342241691
Generation 4 - Current best internal CV score: 0.516328357394
Generation 5 - Current best internal CV score: 0.516996695361
Generation 6 - Current best internal CV score: 0.516996695361
Generation 7 - Current best internal CV score: 0.516996695361
Generation 8 - Current best internal CV score: 0.516996695361
Generation 9 - Current best internal CV score: 0.517000588891
Generation 10 - Current best internal CV score: 0.519342814841
Generation 11 - Current best internal CV score: 0.519342814841
Generation 12 - Current best internal CV score: 0.519342814841


TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: LogisticRegression(RFE(LogisticRegression(input_matrix, C=0.01, dual=False, penalty=l2), criterion=gini, max_features=0.950

In [26]:
optimiser.export("conafor_tpot_v7.py")
out_classes = model.predict(test_features)
true_detections = out_classes == test_classes
print(np.count_nonzero(true_detections)/float(len(true_detections)))

NameError: name 'model' is not defined

In [ ]:
np.count_nonzero(true_detections)/float(len(true_detections))